# Find Spatial Locations
# Buffer a point, return all data within spatial geometry of buffer

In [1]:
# import libraries
import getpass
from arcgis.gis import *
from arcgis import geometry
from arcgis import features


In [2]:
# Utilize getpass for password management in Jupyter notebooks
password = getpass.getpass()

········


In [3]:
# Connect to GIS
# gis = GIS('http://www.arcgis.com', 'abrown_citygov', password)
gis = GIS('https://neenterprise.esri.com/portal', 'abrown', password)
gis

GIS @ https://neenterprise.esri.com/portal version:6.1

In [55]:
# Manually create a point utilizing either decimal degrees or a projected coordinate 
# system (NAD 1983 StatePlane Pennsylvania South FIPS 3702 Feet)
# To properly buffer the boundary, need to pass in State Plane coordinates

# pt_coords = features.feature.Point({"x" : -75.127341, "y" : 39.981295, "spatialReference" : {"wkid" : 4326}})
pt_coords = features.feature.Point({"x" : 2703357.38, "y" : 246961.77, "spatialReference" : {"wkid" : 2272}})

pt = geometry.Point(pt_coords)
feat = features.Feature(geometry=pt, attributes={'OBJECTID': 1,
                                                        'name': 'Test_point',
                                                        'type': 'Feature'})

# Convert the feature into a feature set to utilize in geoprocessing tools
fset = features.FeatureSet([feat])
fset

<FeatureSet> 1 features

In [56]:
# Search for Philadelphia Redacted Crime, put query directly in search
crime_s = gis.content.search("title:PhillyCrime", item_type="Feature Service")
incidents_service = crime_s[0]

incident_layers = incidents_service.layers
incidents = incident_layers[0]

In [58]:
# Create a map object, add layers then visualize

map_1 = gis.map("2143 York Street, Philadelphia, PA 19125")
map_1.add_layer(incidents)
map_1.add_layer(fset)
map_1

MapView(layout=Layout(height='400px', width='100%'))

In [59]:
# Buffer the point using projected coordinate system(NAD 1983 StatePlane Pennsylvania South FIPS 3702 Feet)
# Do not save to permanent layer
point_buff = features.use_proximity.create_buffers(fset.to_dict(), distances=[500], units = 'Feet', 
                                                   context={"wkid" : 2272})
# This option would save the output
#point_buff = features.use_proximity.create_buffers(fset.to_dict(), distances=[500], units = 'Feet', 
#                                                   output_name='Buffer_output', context={"wkid" : 2272})

map_1.add_layer(point_buff)

In [60]:
# Run summarize within tool to look at how many points fall within the boundary.
output = features.analysis.summarize_within(sum_within_layer=point_buff, summary_layer=incidents, sum_shape=True, 
                                   summary_fields=[], group_by_field='text_gener', minority_majority=False, 
                                   percent_shape=False, output_name=None, context={"wkid" : 2272}, gis=None)

In [61]:
# The output of the summarize within did not specifify an output_name, the result is a dictionary.  Grab the dictionary keys
# to separate both the total numbers and grouped values.
print(output)
print(type(output))

# Return just FeatureCollection
final = output.get('result_layer')
related = output.get('group_by_summary')

{'result_layer': <FeatureCollection>, 'group_by_summary': <FeatureCollection>}
<class 'dict'>


In [62]:
# FeatureCollections are different than FeatureLayerCollections; unless one saves the layer using output_name
# a where clause cannot be utilized.
finallayers = final.query()
finallayers

<FeatureSet> 1 features

In [63]:
# Show query results in data table
finallayers.sdf

,AnalysisArea,BUFF_DIST,Join_ID,OBJECTID,ORIG_FID,Point_Count,SHAPE,name,type
0,0.072905,500,1,1,1,217,"{""rings"": [[[-75.12734031650234, 39.9826679230...",Test_point,Feature


In [64]:
# FeatureCollections are different than FeatureLayerCollections; unless one saves the layer using output_name
# a where clause cannot be utilized.
relatedlayers = related.query()
relatedlayers

<FeatureSet> 23 features

In [65]:
# Show query results in data table
relatedlayers.sdf

,Join_ID,OBJECTID,Point_Count,objectid_1,text_gener
0,1,1,2,1,Aggravated Assault Firearm
1,1,2,6,2,Aggravated Assault No Firearm
2,1,3,42,3,All Other Offenses
3,1,4,1,4,Arson
4,1,5,3,5,Burglary Non-Residential
5,1,6,15,6,Burglary Residential
6,1,7,5,7,Disorderly Conduct
7,1,8,10,8,DRIVING UNDER THE INFLUENCE
8,1,9,2,9,Forgery and Counterfeiting
9,1,10,10,10,Fraud


In [54]:
# Create a second map object for clarity
map_2 = gis.map("2143 York Street, Philadelphia, PA 19125")
map_2.add_layer(finallayers)
map_2

MapView(layout=Layout(height='400px', width='100%'))